# Hyperparameter Tuning using HyperDrive


In [27]:

# import dependencies
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset

from azureml.core import ScriptRunConfig

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


In [28]:
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="capstone-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-131737
Azure region: southcentralus
Subscription id: b968fb36-f06a-4c76-a15f-afab68ae7667
Resource group: aml-quickstarts-131737


## Dataset

In [29]:
# NOTE: update the key to match the dataset name
found = False
key = "diabetes-classification"
description_text = "Diabetes datasets for capstone project"

if key in ws.datasets.keys(): 
        found = True        
        dataset = ws.datasets[key] 

if not found:
        # Create Hyperdrive Dataset and register it into Workspace
        url = 'https://raw.githubusercontent.com/purunep/Capstone/main/project/data/diabetes.csv'
        dataset = Dataset.Tabular.from_delimited_files(url)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

from train import clean_data
[x_data, y_data] = clean_data(dataset)

In [31]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cpu_cluster_name = "puru-compute"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D12_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [35]:
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
import joblib

In [41]:
env = Environment('puru-environment')
print(env)

Environment(Name: puru-environment,
Version: None)


In [44]:
!pip install --upgrade azureml-train-automl-client

Requirement already up-to-date: azureml-train-automl-client in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (1.19.0)
     |████████████████████████████████| 53 kB 1.9 MB/s eta 0:00:011
ERROR: azureml-contrib-notebook 1.19.0 has requirement nbconvert<6, but you'll have nbconvert 6.0.7 which is incompatible.
ERROR: azure-cli-core 2.16.0 has requirement msal~=1.0.0, but you'll have msal 1.6.0 which is incompatible.
ERROR: azure-cli-core 2.16.0 has requirement msal-extensions~=0.1.3, but you'll have msal-extensions 0.2.2 which is incompatible.
  Attempting uninstall: pyopenssl
    Found existing installation: pyOpenSSL 20.0.0
    Uninstalling pyOpenSSL-20.0.0:
      Successfully uninstalled pyOpenSSL-20.0.0


In [45]:
from azureml.core import Environment
envit = Environment.get(workspace=ws,name='puru-environment')

Exception: Error retrieving the environment definition. Code: 404
: {
  "error": {
    "code": "UserError",
    "severity": null,
    "message": "No definitions exist for environment puru-environment",
    "messageFormat": null,
    "messageParameters": null,
    "referenceCode": null,
    "detailsUri": null,
    "target": null,
    "details": [],
    "innerError": {
      "code": "NotFoundError",
      "innerError": null
    },
    "debugInfo": null
  },
  "correlation": {
    "operation": "7e0e64b2c8c3134ba25f31a2c6720cb4",
    "request": "c9cb1bac60953446"
  },
  "environment": "southcentralus",
  "location": "southcentralus",
  "time": "2020-12-25T16:48:43.5662567+00:00",
  "componentName": "environment-management"
}

In [39]:

# Specify parameter sampler
ps = RandomParameterSampling( {
        "--C": uniform(0.50,1.00),
       "--max_iter" : choice(10,20,30)
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)



#sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='./conda_dependencies.yml')
#envit = Environment.get(workspace=ws,name='myenv')

config  = ScriptRunConfig(source_directory='./',
                         script='train.py',                         
                         compute_target=cpu_cluster,
                         environment=env)



#hyperdrive_run_config = <your config here>
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
                                   hyperparameter_sampling = param_sampling,
                                   primary_metric_name = 'Accuracy',
                                   max_total_runs = 50,
                                   max_concurrent_runs = 4,
                                   primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                   policy = policy,
                                   run_config = config)

#estimator=estimator use to replace estimator in hyperdriveconfig

In [40]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()


print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = <your policy here>

#TODO: Create the different params that you will be using during training
param_sampling = <your params here>

#TODO: Create your estimator and hyperdrive config
estimator = <your estimator here>

hyperdrive_run_config = <your config here?

In [ ]:
#TODO: Submit your experiment

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service